In [1]:
# Librerias.

import pandas as pd
from sklearn.neighbors import NearestNeighbors
from tqdm import tqdm


Mediante el método del vecino más cercano, adjudicamos los polígonos a los diferentes puertos.

In [6]:
# Cargar datos.
df_cuota_pesquera = pd.read_csv(r'ruta a cuota_pesquera.csv')
df_caladeros = pd.read_csv(r'ruta a coordenadas_caladeros.csv')

# Calcular la distancia entre todos los caladeros y puertos.
nbrs = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(df_cuota_pesquera[['Latitud', 'Longitud']])
distances, indices = nbrs.kneighbors(df_caladeros[['Latitud', 'Longitud']])

# Preparar tqdm.
total_caladeros = len(df_caladeros)
with tqdm(total=total_caladeros, desc='Procesando') as pbar:
    # Asignar el puerto más cercano a cada caladero
    df_caladeros['Puerto_asociado'] = df_cuota_pesquera.iloc[indices.flatten()]['Puerto'].values
    pbar.update(total_caladeros)
    
# Eliminar duplicados de los caladeros asociados a cada puerto.
df_resultado_clorofila_caladeros = df_caladeros.drop_duplicates(subset=['Puerto_asociado', 'Caladero'])

# Agrupar caladeros por puerto y generar lista de caladeros asociados.
caladeros_por_puerto = df_resultado_clorofila_caladeros.groupby('Puerto_asociado')['Caladero'].apply(list).reset_index()

# Renombrar columnas.
caladeros_por_puerto.columns = ['Puerto', 'Caladeros_asociados']

# Guardar resultados.
caladeros_por_puerto.to_csv('caladeros_por_puerto.csv', index=False)


Procesando: 100%|██████████| 25058/25058 [00:00<00:00, 1439599.90it/s]
